In [1]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import os
from functools import reduce
from itertools import chain
from sympy import factorint
import glob
import pyDOE2
from sklearn.utils import shuffle


In [10]:
num_rows = 10
num_cols = 10

available_rows = range(num_rows)
available_cols = range(num_cols)

wells = list(itertools.product(*[available_cols,available_rows]))
wells = pd.DataFrame(wells,columns=['col','row'])

wells_per_plate = len(wells)
wells

wells['droplets'] = (9-wells['col'])
# wells['droplets'] = 5
wells

,col,row,droplets
0,0,0,9
1,0,1,9
2,0,2,9
3,0,3,9
4,0,4,9
...,...,...,...
95,9,5,0
96,9,6,0
97,9,7,0
98,9,8,0


In [11]:
def snake_df(df):
    sorted_array = []
    for row, row_df in df.groupby('row'):
        if row % 2 == 0:
            row_df = row_df.sort_values('col',ascending=True)
        else:
            row_df = row_df.sort_values('col',ascending=False)
        sorted_array.append(row_df)
    
    return pd.concat(sorted_array)


sorted_df = snake_df(wells)
sorted_df

,col,row,droplets
0,0,0,9
10,1,0,8
20,2,0,7
30,3,0,6
40,4,0,5
...,...,...,...
49,4,9,5
39,3,9,6
29,2,9,7
19,1,9,8


In [12]:
name = '10x10_9to0'
file_path = f'./{name}.csv'
file_path
sorted_df = sorted_df.rename(columns={'row':'Row','col':'Column','droplets':'Droplet'})
df = sorted_df.copy()

df['Droplet'] = df['Droplet'].astype(int)
df = df[df['Droplet'] != 0]

print(file_path)
df[['Row','Column','Droplet']].set_index('Row').to_csv(file_path)

./10x10_9to0.csv


In [17]:
start_0 = pd.read_csv('./10x10_0to9.csv')
start_9 = pd.read_csv('./10x10_9to0.csv')
comb = start_0.set_index(['Row','Column']).join(start_9.set_index(['Row','Column']),lsuffix='_0',rsuffix='_1')
comb['Total'] = comb['Droplet_0'] + comb['Droplet_1']
comb['Total'].describe()

count    80.0
mean      9.0
std       0.0
min       9.0
25%       9.0
50%       9.0
75%       9.0
max       9.0
Name: Total, dtype: float64

In [8]:
# for plate, plate_df in sorted_df.groupby('plate'):
#     for [name,conc],df in plate_df.groupby(['reagent','stock']):
#         plate_name = experiment_id + '_' + str(plate)
#         dir_path = r'C:\Users\conar\printing-platform\Print_arrays\{}'.format(plate_name)

#         file_name = '{}\{}-{}.csv'.format(dir_path,name,round(conc,2))
#         df = df.rename(columns={'row':'Row','col':'Column','droplets':'Droplet'})
#         df['Droplet'] = df['Droplet'].astype(int)
#         df = df[df['Droplet'] != 0]
#         if len(df) == 0:
#             print(f'{name}-{conc} has no droplets')
#             continue
#         print(file_name)
#         df[['Row','Column','Droplet']].set_index('Row').to_csv(file_name)